In [2]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random

import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

import ant_vocab
import sym_vocab

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성 
def crawl_news_summy(url, raw_txt_name, summy_txt_name, num_word):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en')
    news.download()
    # HTML 파싱 적용
    news.parse()
    # 지정된 단어 갯수를 기준으로 textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_word) # 800자 기준으로 하기
    title = news.title
    title = title.lower()
    
    title = re.sub('\([^)]*\)', '', title) # 괄호 안 제거 
    
    # Textrank를 적용하여, 요약 지문에 해당하는 영역을 별도로 추출 
    summy_text = summarize(news.text, word_count = 50)
    
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: 
        f.write(text)
    
    text = re.sub('\n', ' ', text)
    # 크롤링한 raw data를 별도로 저장 
    with open(summy_txt_name, 'w', encoding = 'utf-8') as f:
        f.write(summy_text)
        
    summy_text = re.sub('\n', ' ', summy_text)
    
    
    return text, title, summy_text

In [8]:
# 빈칸 지문 생성을 위한 함수를 별도로 정의
def bf_create_long_summy(summy_txt_name, prepro_summy_name):
    # 문장 번호: 문장 형식의 저장을 위해 변수 생성
    test_dict = {}
    
    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(summy_txt_name, 'r', encoding = 'utf-8') as f:
    # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text
    # 전처리된 데이터를 별도의 파일로 저장
    with open(prepro_summy_name, 'w', encoding = 'utf-8') as f: # 중첩 형식 사용해보기
        # 줄 바꿈을 기준으로 문장을 구분하여 저장
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [18]:
def shake_bot_long(filename, valid_article_path):
    # 설정된 세팅 값 불러오기
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
        
    # 실행시 최초의 상태로 초기화하여 모델 재사용이 가능하도록 함
    tf.reset_default_graph()
    # 학습된 단어 사전과 본문 최대 길이와 요약 최대 길이 설정 값을 불러옴
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    # 불러온 설정 값에 따라서 검증 데이터셋을 생성
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", vaild_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        # 학습된 체크포인트와 모델을 불러오기 
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True) # 검증 단계
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # 검증을 진행할 배치 사이즈 설정
        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
        
        # 검증 단계인 만큼 인코더 인풋 부분만 고려하여 학습 진행
        print("셰봇이 수능 요약 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            # 문장 -> 문장 예측이 가능하도록 데이터 분할 
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]
            # 검증을 위한 배치에 따라 모델 및 데이터 셋 설정 준비 
            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }
            # Prediction을 목적으로 학습된 모델을 검증을 위한 설정 값을 적용하여 예측 결과 생성
            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            # 인덱스 -> 단어 사전을 통해 학습된 결과를 단어로 표현.
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            
            # 생성된 예측 결과물을 문장 단위로 저장 
            with open(filename, "w") as f:
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))
        
        


In [13]:
# 학습 결과물 중 <unk> 부분 처리
def delete_unk(content, unk_sentence):
    
    container = []
    
    text = open(content, 'r', encoding = 'utf-8').read()
    
    # 각 단어에 대해서 품사 태깅
    tagged_list = pos_tag(word_tokenize(text))
    
    # 명사 부분만 활용
    nnp_list = [t[0] for t in tagged_list if (t[1] == "NNP") or (t[1] == "NN")]
    
    fd_names = FreqDist(nnp_list)
    
    # 빈도수 기반으로 Unk 처리하기
    freq = fd_names.most_common(2)
    
    #for tagged in tagged_list:
        
        #if tagged[1] == 'NN' or 'NNP':
            
            #container.append(tagged[0]) # 단어만 추출
    
    
    # unk 부분을 처리하는 방법 중 하나로 최고 빈도수를 기반으로 하는 방식
    ## 현재 여기서는 정확도를 해치는 것으로 판단 
    #replace_sentence = re.sub('< unk >', freq[0][0], unk_sentence)
    
    # 다중 숫자에 대한 표현은 Several로 대체 
    replace_sentence = re.sub('# nd', 'Several', unk_sentence)
    
    #replace_sentence = re.sub('# nd', 'Several', replace_sentence)
    
    # 불필요한 줄 바꿈표 제거 
    replace_sentence = re.sub('\n', '', replace_sentence)
    
    return replace_sentence
    

In [21]:
# 요약 지문 생성
url = 'https://www.nytimes.com/interactive/2019/11/15/nyregion/amazon-mechanical-turk.html?action=click&module=Top%20Stories&pgtype=Homepage'
num_word = 400
raw_txt_name = 'CSAT_40.txt'
summy_txt_name = 'summy_40_txt'
prepro_summy_name = 'prepro_summy_40.txt'
learned_txt_name = 'summy_40.txt'

def summy_40(url, raw_txt_name, summy_txt_name, prepro_summy_name, learned_txt_name, num_word):
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기
    text, title, summy_text = crawl_news_summy(url, raw_txt_name, summy_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_long_summy(summy_txt_name, prepro_summy_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_long(learned_txt_name, prepro_summy_name) # 셰봇이 학습시키기 (자연어 요약)
    
    summy_list = list()
    
    v_list = list()
    
    # 요약 지문에 대해서 parapharasing 적용
    with open(learned_txt_name, 'r', encoding = 'utf-8') as f:
        
        for summy_text in f.readlines():
            
            summy_text = delete_unk(prepro_summy_name, summy_text)
            
            token = word_tokenize(summy_text)
            
            tag = pos_tag(token)
            
            
            # 요약 지문에 대해서 동사 부분 빈칸 처리
            for ix, t in enumerate(tag): 
                
                if ((t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")):
                        
                    v_list.append(t[0])
                    
                    if len(v_list) == 1:
                    
                        token[ix] = '___(A)___'
                    
                    elif len(v_list) == 2:
                        
                        token[ix] = '___(B)___'
                        
                        
                if len(v_list) == 2:
                    
                    break 
                    
            token = ' '.join(token)
            
            summy_text = token.capitalize() ## 요약 지문 완성
            
    
    # 정답 지문 생성 
    sym_words_list = list()
    
    for v in v_list:
        # 유의어 사전을 활용
        sym_dict = sym_vocab.crawling_dict(v)
        
        sym_list = sym_dict[v]
        # 유의어가 존재하는 경우
        if sym_list == list:
        
            random.shuffle(sym_list)
            
            sym_words_list.append(sym_list[0])
        # 유의어가 존재하지 않는 경우
        elif sym_list != list:
            
            sym_words_list.append(sym_list)
        
    
    # 오답 지문 생성
    ant_words_list = list()
    
    for v in v_list:
        
        ant_word = ant_vocab.crawling_dict(v)
        
        ant_words_list.append(ant_word)
    
        
    
    question_list = list()       
    # 정답 부분
    answer = '(A): ' + v_list[0] + ' ' + '(B): ' + v_list[1]
    print(answer)
    
    # 정답 이외의 나머지 4개 선지 제작 
    question_list.append(answer)
    
    question_1 = '(A): ' + sym_words_list[1] + ' ' + '(B): ' + ant_words_list[1]
    
    question_list.append(question_1) 
    
    question_2 = '(A): ' + sym_words_list[0] + ' ' + '(B): ' + ant_words_list[0]
    
    question_list.append(question_2) 
    
    question_3 = '(A): ' + ant_words_list[1] + ' ' + '(B): ' + sym_words_list[0]
    
    question_list.append(question_3) 
    
    question_4 = '(A): ' + ant_words_list[1] + ' ' + '(B): ' + sym_words_list[0]
    
    question_list.append(question_4) 
       
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('40. 다음 글의 내용을 한 문장으로 요약하고자 한다. 빈칸 (A), (B)에 들어갈 말로 가장 적절한 것은?')
    print('-----------------------------------')
    print(text) # 지문 600자
    print('-----------------------------------')
    print(summy_text)
    print('\n')
    for question in question_list:
        print(question)
    
if __name__ == "__main__":
    
    summy_40(url, raw_txt_name, summy_txt_name, prepro_summy_name, learned_txt_name, num_word)  


0 it works like this: employers , known as requesters , post batches of what are called human intelligence tasks , or hits , on mechanical turk 's website
1 while amazon pays all of its american employees at least $## an hour and favors raising the minimum wage , the company declined numerous requests to comment about the pay policy for turkers or anything else regarding mechanical turk
셰봇이 단어 사전 불러오는 중...
셰봇이 기본 설정 값 초기화 중...
Loading saved model...
INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt-297190
셰봇이 수능 문제 생성 준비 중.. summy_40.txt...
셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. summy_40.txt...
(A): asked (B): raise




40. 다음 글의 내용을 한 문장으로 요약하고자 한다. 빈칸 (A), (B)에 들어갈 말로 가장 적절한 것은?
-----------------------------------
I had just earned another 5 cents on a digital work marketplace run by Amazon called Mechanical Turk. It works like this: Employers, known as requesters, post batches of what are called Human Intelligence Tasks, or HITs, on Mechanical Turk’s website. ⚙️ Try Your Hand